1. https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.379.6473&rep=rep1&type=pdf
2. http://yifanhu.net/PUB/cf.pdf

In [52]:
import os
import sys
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, save_npz, load_npz, vstack, hstack, lil_matrix
import implicit
import pickle
from implicit.evaluation import train_test_split, precision_at_k, mean_average_precision_at_k

In [53]:
def load_data(file_name):
    data = pd.read_csv(file_name)
    data['URL_display'] = data['URL_display'].apply(lambda x: str(x))
    new_df = data[['gtid','URL_display']]
    new_df.dropna(how='all')
    
    return new_df

In [54]:
data = load_data('viewlog.csv')

data_sub = data
data.head()

,gtid,URL_display
0,[-Je_3wrnuqiItrtyTKzblIjLxOS_-EfUMdnxzMriWVE1p...,https://www.marsflag.com/ja/
1,[-Jir8X21sNsC8euLWfkc988pcZox5NMfE0yZZqCcfIWq5...,https://www.marsflag.com/ja/
2,[-Jir8X21sNsd7gI6KPoc7s8pcZox5NMfE0yZZqCcfIVFH...,https://www.marsflag.com/ja/
3,[-Jir8X21sNsd7gI6KPoc7s8pcZox5NMfE0yZZqCcfIVFH...,https://www.marsflag.com/ja/
4,[-OyNSvDMS6PjK-0U9GVl8s8pcZox5NMfXxkzmbfZsz5vf...,https://www.marsflag.com/ja/


In [55]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18396 entries, 0 to 18395
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   gtid         18396 non-null  object
 1   URL_display  18396 non-null  object
dtypes: object(2)
memory usage: 287.6+ KB


In [56]:
unique_url = data.URL_display.unique().tolist()
unique_user = data.gtid.unique().tolist()


len(unique_user),len(unique_url)

(13666, 95)

In [57]:
unique_user.index(str('[-Je_3wrnuqiItrtyTKzblIjLxOS_-EfUMdnxzMriWVE1p4FPVcRW0g..]'))

0

In [58]:
data.head()

,gtid,URL_display
0,[-Je_3wrnuqiItrtyTKzblIjLxOS_-EfUMdnxzMriWVE1p...,https://www.marsflag.com/ja/
1,[-Jir8X21sNsC8euLWfkc988pcZox5NMfE0yZZqCcfIWq5...,https://www.marsflag.com/ja/
2,[-Jir8X21sNsd7gI6KPoc7s8pcZox5NMfE0yZZqCcfIVFH...,https://www.marsflag.com/ja/
3,[-Jir8X21sNsd7gI6KPoc7s8pcZox5NMfE0yZZqCcfIVFH...,https://www.marsflag.com/ja/
4,[-OyNSvDMS6PjK-0U9GVl8s8pcZox5NMfXxkzmbfZsz5vf...,https://www.marsflag.com/ja/


In [59]:
data['URL_display'] = data['URL_display'].apply(lambda x : unique_url.index(str(x)))
data['gtid'] = data['gtid'].apply(lambda x : unique_user.index(str(x)))

data.head()

,gtid,URL_display
0,0,0
1,1,0
2,2,0
3,2,0
4,3,0


In [60]:
def sparse_matrices(df):
    alpha = 40
    sparse_user_item = csr_matrix( ([alpha]*len(df['URL_display']), (df['gtid'], df['URL_display']) ))
    
    
    sparse_item_user = sparse_user_item.T.tocsr()

    return sparse_user_item, sparse_item_user

In [61]:
sparse_user_item, sparse_item_user = sparse_matrices(data)

In [62]:
import implicit


model = implicit.als.AlternatingLeastSquares(factors=50)

model.fit(sparse_item_user)

In [63]:
# recommend items for a user
user_items = sparse_item_user.T.tocsr()
recommendations = model.recommend(0, user_items)

# find related items
related = model.similar_items(2)

In [64]:
recommendations

[(45, 0.0034509674),
 (32, 0.0030989796),
 (6, 0.002794236),
 (28, 0.002765648),
 (50, 0.0022764448),
 (8, 0.0022593215),
 (53, 0.0022011772),
 (1, 0.0021570586),
 (36, 0.0021513328),
 (19, 0.0021120738)]

In [65]:
related

[(2, 1.0),
 (81, 0.30667186),
 (85, 0.24135548),
 (41, 0.22277386),
 (89, 0.18267319),
 (47, 0.18080696),
 (13, 0.15608345),
 (17, 0.15460272),
 (54, 0.15150985),
 (29, 0.14820807)]

In [66]:
# !pip install pyspark -q

In [67]:
data_sub.head()

,gtid,URL_display
0,0,0
1,1,0
2,2,0
3,2,0
4,3,0


In [68]:
data_sub = data_sub.groupby(['gtid','URL_display']).size().reset_index().rename(columns={0:'count'})
data_sub.head()

,gtid,URL_display,count
0,0,0,1
1,1,0,1
2,2,0,2
3,3,0,1
4,4,0,1


In [69]:
data_sub['URL_display'] = data_sub['URL_display'].apply(lambda x : unique_url.index(str(x)))
data_sub['gtid'] = data_sub['gtid'].apply(lambda x : unique_user.index(str(x)))


ValueError: '0' is not in list

In [70]:

data_sub.head(15)

,gtid,URL_display,count
0,0,0,1
1,1,0,1
2,2,0,2
3,3,0,1
4,4,0,1
5,5,0,1
6,6,0,1
7,7,0,1
8,8,0,1
9,9,1,1
